0. Imports

In [13]:
import pandas as pd
import numpy as np

1. Load csv

In [14]:
# change to your file location
df = pd.read_csv('/content/drive/MyDrive/Škola/DM/spravanie_zamestnancov_v_zavislosti_od_casu_IV/MLM_ZAM_stats.csv', ';', usecols=range(0,10))
df.head(10)

,Effect,Intercept,HODINA,HODINA_STV,PO,UT,STR,STVR,PIA,SO
0,uvod_ZAM,5.58384,-0.68231,0.02633,0.16019,0.00955,-0.76405,-0.98585,0,0
1,studium_ZAM,0.92959,0.06409,-0.00612,0.38236,0.82227,-0.17730,-0.00245,0,0
2,oznamy_ZAM,-3.91218,0.40371,-0.02517,2.31863,2.96682,0.57197,-0.49513,0,0


2. Create collection of weekdays

In [15]:
days = ['PO', 'UT', 'STR', 'STVR', 'PIA']

3. Create estimates for web parts

In [16]:
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
index = 0

# Cycle through hours from 7 to 23
for x in range (7,23):
      new_row_uvod = {}
      new_row_studium = {}
      new_row_oznamy = {}
      i = 1

      # Cycle through weekdays
      for day in days:

        # Create logits estimates
        logit_uvod = df.at[index, 'Intercept'] + df.at[index, 'HODINA']*x+df.at[index, 'HODINA_STV']*(x*x)+df.at[index, day]
        logit_studium = df.at[index+1, 'Intercept'] + df.at[index+1, 'HODINA']*x+df.at[index+1, 'HODINA_STV']*(x*x)+df.at[index+1, day]
        logit_oznamy = df.at[index+2, 'Intercept'] + df.at[index+2, 'HODINA']*x+df.at[index+2, 'HODINA_STV']*(x*x)+df.at[index+2, day]
        
        reference_web = 1 / (1 + np.exp(logit_uvod) + np.exp(logit_studium) + np.exp(logit_oznamy))
        
        # Create estimates for web parts
        estimate_uvod = np.exp(logit_uvod) * reference_web
        estimate_studium = np.exp(logit_studium) * reference_web
        estimate_oznamy = np.exp(logit_oznamy) * reference_web

        den = str(i) + '_' + day

        # Create new rows and append it to dataframe
        new_row_uvod.update({den: estimate_uvod})
        new_row_studium.update({den: estimate_studium})
        new_row_oznamy.update({den: estimate_oznamy})
        i = i + 1
      
      # Append time to rows
      new_row_uvod.update({'0_hod': x})
      new_row_studium.update({'0_hod': x})
      new_row_oznamy.update({'0_hod': x})

      # Update dataframes
      df1 = df1.append(new_row_uvod, sort=False, ignore_index=True)
      df2 = df2.append(new_row_studium, sort=False, ignore_index=True)
      df3 = df3.append(new_row_oznamy, sort=False, ignore_index=True)

df1.head()

,0_hod,1_PO,2_UT,3_STR,4_STVR,5_PIA
0,7.0,0.643090,0.488900,0.590103,0.503968,0.728416
1,8.0,0.579029,0.421583,0.525251,0.439083,0.673743
2,9.0,0.530193,0.374092,0.476327,0.391748,0.629453
3,10.0,0.498869,0.345391,0.444466,0.361387,0.598877
4,11.0,0.485519,0.333875,0.429486,0.346815,0.583662


4. Export to excel

In [17]:
# Creating Excel Writer Object from Pandas  
writer = pd.ExcelWriter('ZAM_accessing_the_web_parts.xlsx',engine='xlsxwriter')   
workbook=writer.book
worksheet=workbook.add_worksheet('ZAM')
writer.sheets['ZAM'] = worksheet

# Úvod
worksheet.write(0, 0, "Úvod")
df1.to_excel(writer, sheet_name='ZAM',startrow=1 , startcol=0, index=False)

# Śtúdium
worksheet.write(0, 7, "Štúdium")
df2.to_excel(writer, sheet_name='ZAM',startrow=1 , startcol=7, index=False)

# Oznamy
worksheet.write(0, 14, "Oznamy")
df3.to_excel(writer, sheet_name='ZAM',startrow=1 , startcol=14, index=False)
writer.save()